In [22]:
## Output shows stuff to display on FrontEnd


import time

from flask import Flask, request, jsonify

from sklearn.feature_extraction.text import TfidfVectorizer ## TFIDF calculation
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction import text

import nltk
from nltk.stem.wordnet import WordNetLemmatizer ## pos tagging and lemmatization
from nltk.corpus import wordnet
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import RegexpTokenizer ## Tokenizer which removes punctuation

from langdetect import detect ## Detect Language
import pickle ## pickle the model
import scipy.sparse ## Convert sparse matrix to SciPy CSC matrix .npz

#import numpy as np



## TO BE DONE AS SOON AS SERVER STARTS RUNNING i.e description_array file should be kept open and running even when nobody is pinging on enter description
## Load full description array
description_array = pickle.load(open("description_array.pickle", "rb"))
## Keep max_features the same as that training the model
vectorizer = TfidfVectorizer(stop_words = 'english', max_features = 9500)   
## tokenize and build vocabulary
vectorizer.fit(description_array)

## Load all rating array
rating_array = pickle.load(open("rating_array.pickle", "rb"))
#print (rating_array[0])

## Load all track name array
track_name_array = pickle.load(open("track_name_array.pickle", "rb"))
#print (track_name_array[0])

## Load rating count for each description
rating_count_array = pickle.load(open("rating_count_tot.pickle", "rb"))

## Load age group for each description
age_group_array = pickle.load(open("age_group.pickle", "rb"))

## Load prime genre for each description
genre = pickle.load(open("genre.pickle", "rb"))

## Load All Documents Encoded Sparse Array npz file
sparse_matrix = scipy.sparse.load_npz('sparse_matrix_actual.npz')
all_documents_encoded = sparse_matrix.todense()
# Sometimes sparse_matrix.todense() shows memory error. But trying after some time it doesnt show error




## Test Description - Pos tagging and lemmatization with nltk:
## POS Tagging and Lemmatization function      
## See 'pos_tagging_and_lemmatization' file full explanation       
regexp_tokenizer = RegexpTokenizer(r'\w+')        
def lemmatizeDescription(current_description):
    ## No need to include stop words restriction here ad TfidfVectorizer(stop_words = 'english') is added as vectorizer
    current_description.lower()
    tokenized_description = regexp_tokenizer.tokenize(current_description)
    tag = nltk.pos_tag(tokenized_description)
    lemmatized_list = []
    for i in range(len(tag)):
        lemmatized_list.append(lemmatizer.lemmatize(tag[i][0],get_wordnet_pos(tag[i][1])))
    lemmatized_Description = " ".join(lemmatized_list)
    return lemmatized_Description

def get_wordnet_pos(treebank_pos):

    if treebank_pos.startswith('J'):
        return wordnet.ADJ
    elif treebank_pos.startswith('V'):
        return wordnet.VERB
    elif treebank_pos.startswith('N'):
        return wordnet.NOUN
    elif treebank_pos.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
        #return ''        

        

        
def sortTopXpercent(all_documents_similarity, topXpercent):

    x = 1
    while(x<=topXpercent):

        for i in range(len(all_documents_similarity)-1, -1, -1):
            
            if(i == 0):
                continue
            if(all_documents_similarity[i]>all_documents_similarity[i-1]):
                temp = all_documents_similarity[i]
                all_documents_similarity[i] = all_documents_similarity[i-1]
                all_documents_similarity[i-1] = temp
                
        x = x+1

    all_documents_similarity_sorted_topXpercent = all_documents_similarity[:topXpercent]

    return all_documents_similarity_sorted_topXpercent
           
        
        
        
## For the graph of "number of users" by "Age Group"
## Helper function to add new values into users_by_ageGroup_dict according to age group percentages in users_by_ageGroup_dict_percent, for the current document_weight
def age_group_percent(users_by_ageGroup_dict, users_by_ageGroup_dict_percent, document_weight):
    for x in users_by_ageGroup_dict:
        temp3 = users_by_ageGroup_dict[x]
        users_by_ageGroup_dict[x] = int(temp3 + (document_weight*users_by_ageGroup_dict_percent[x]))
    return users_by_ageGroup_dict        

def users_by_ageGroup_dict_modify_by_percentage(age_group, users_by_ageGroup_dict, document_weight):
    if(age_group == "4+" or age_group == None or age_group == "0"):
        users_by_ageGroup_dict_percent = { "Children_5-13":0.05, "Teenager_13-18":0.25, "Adult_18-50":0.55, "Elderly_50+":0.15}
        users_by_ageGroup_dict = age_group_percent(users_by_ageGroup_dict, users_by_ageGroup_dict_percent, document_weight)     
    elif(age_group == "9+"):
        users_by_ageGroup_dict_percent = { "Children_5-13":0.03, "Teenager_13-18":0.27, "Adult_18-50":0.55, "Elderly_50+":0.15}
        users_by_ageGroup_dict = age_group_percent(users_by_ageGroup_dict, users_by_ageGroup_dict_percent, document_weight)        
    elif(age_group == "12+"): 
        users_by_ageGroup_dict_percent = { "Children_5-13":0.001, "Teenager_13-18":0.25, "Adult_18-50":0.599, "Elderly_50+":0.15}
        users_by_ageGroup_dict = age_group_percent(users_by_ageGroup_dict, users_by_ageGroup_dict_percent, document_weight)        
    elif(age_group == "17+"): 
        users_by_ageGroup_dict_percent = { "Children_5-13":0.0, "Teenager_13-18":0.05, "Adult_18-50":0.8, "Elderly_50+":0.15}
        users_by_ageGroup_dict = age_group_percent(users_by_ageGroup_dict, users_by_ageGroup_dict_percent, document_weight)
    return users_by_ageGroup_dict        
        
        
        
        
## For the equalized graph of "number of users" by "rating"
## Helper function to add new values into users_by_rating_equalized_dict according to rating percentages in users_by_rating_equalized_dict_percent, for the current document_weight
def rating_percent(users_by_rating_equalized_dict, users_by_rating_equalized_dict_percent, document_weight):
    for x in users_by_rating_equalized_dict:
        temp1 = users_by_rating_equalized_dict[x]
        users_by_rating_equalized_dict[x] = int(temp1 + (document_weight*users_by_rating_equalized_dict_percent[x])) 
    return users_by_rating_equalized_dict        
## For equalized 'number of users' - apple app store lets you rate with integers from 1 - 5 
def users_by_rating_equalized_dict_modify_by_percentage(document_rating, users_by_rating_equalized_dict, document_weight):
    
    if(document_rating == "1" or document_rating == "1.0"):
        users_by_rating_equalized_dict_percent = { "1.0":1, "2.0":0, "3.0":0, "4.0":0, "5.0":0 }
        users_by_rating_equalized_dict = rating_percent(users_by_rating_equalized_dict, users_by_rating_equalized_dict_percent, document_weight)     
    elif(document_rating == "1.5"):
        users_by_rating_equalized_dict_percent =  { "1.0":0.5, "2.0":0.5, "3.0":0, "4.0":0, "5.0":0 }
        users_by_rating_equalized_dict = rating_percent(users_by_rating_equalized_dict, users_by_rating_equalized_dict_percent, document_weight)     
    elif(document_rating == "2" or document_rating == "2.0"):
        users_by_rating_equalized_dict_percent =  { "1.0":0.25, "2.0":0.5, "3.0":0.25, "4.0":0, "5.0":0 }
        users_by_rating_equalized_dict = rating_percent(users_by_rating_equalized_dict, users_by_rating_equalized_dict_percent, document_weight)     
    elif(document_rating == "2.5"):
        users_by_rating_equalized_dict_percent =  { "1.0":0.2, "2.0":0.3, "3.0":0.3 ,"4.0":0.2, "5.0":0 }
        users_by_rating_equalized_dict = rating_percent(users_by_rating_equalized_dict, users_by_rating_equalized_dict_percent, document_weight)     
    elif(document_rating == "3" or document_rating == "3.0"):
        users_by_rating_equalized_dict_percent =  { "1.0":0.05, "2.0":0.15, "3.0":0.5, "4.0":0.15, "5.0":0.05 }
        users_by_rating_equalized_dict = rating_percent(users_by_rating_equalized_dict, users_by_rating_equalized_dict_percent, document_weight)     
    elif(document_rating == "3.5"):
        users_by_rating_equalized_dict_percent =  { "1.0":0, "2.0":0.2, "3.0":0.3, "4.0":0.3, "5.0":0.2 }
        users_by_rating_equalized_dict = rating_percent(users_by_rating_equalized_dict, users_by_rating_equalized_dict_percent, document_weight)     
    elif(document_rating == "4" or document_rating == "4.0"):
        users_by_rating_equalized_dict_percent =  { "1.0":0, "2.0":0, "3.0":0.25, "4.0":0.5, "5.0":0.25 }
        users_by_rating_equalized_dict = rating_percent(users_by_rating_equalized_dict, users_by_rating_equalized_dict_percent, document_weight)     
    elif(document_rating == "4.5"):
        users_by_rating_equalized_dict_percent =  { "1.0":0, "2.0":0, "3.0":0, "4.0":0.5, "5.0":0.5 }
        users_by_rating_equalized_dict = rating_percent(users_by_rating_equalized_dict, users_by_rating_equalized_dict_percent, document_weight)     
    elif(document_rating == "5" or document_rating == "5.0"):
        users_by_rating_equalized_dict_percent =  { "1.0":0, "2.0":0, "3.0":0, "4.0":0, "5.0":1 }
        users_by_rating_equalized_dict = rating_percent(users_by_rating_equalized_dict, users_by_rating_equalized_dict_percent, document_weight)     
    return users_by_rating_equalized_dict        
        
        
        
        
        
def find_similarity(test_description_modified):
    ## make sparse array of filtered test description
    test_document_vector = vectorizer.transform([test_description_modified])
    test_document_encoded = (test_document_vector.toarray()) 

    #print (test_document_encoded)

    ## Cosine Similarity:

    all_documents_similarity = []
    ## all_documents_similarity is a array in whcih we save the similarity and the primary key/index together as we will have to sort the       list for selecting top similar descriptions, so we need to save the indexes as well
    for i in range(len(all_documents_encoded)):
        all_documents_similarity.append([cosine_similarity(all_documents_encoded[i],test_document_encoded), i])
        
        
#     ## Sort all similarities in desc order    
#     all_documents_similarity_sorted = sorted(all_documents_similarity, reverse = True) 

    ## Select similar documents:
    ## Select Top X% of the sorted values
    ## NOTE - Is there a better way to decide what percentage to select, other than trial and error on percentages?
    Xpercent = 0.15 ##Top 10 documents
    topXpercent = int(len(all_documents_similarity)*(Xpercent/100))
    
#     all_documents_similarity_sorted_topXpercent = all_documents_similarity_sorted[:topXpercent]
#     #print (all_documents_similarity)
#     #print (all_documents_similarity_sorted)
#     #print (topXpercent)
#     #print (all_documents_similarity_sorted_topXpercent)
    
   
    ## SORT ONLY THOSE MUCH SIMILARITIES THAT ARE NEEDED - all_documents_similarity_sorted - dont sort all 7000 or so entires - Sort only to get the topXpercent similarity values
    all_documents_similarity_sorted_topXpercent = sortTopXpercent(all_documents_similarity, topXpercent)
    
    
    #print ("\nRating of most similar app:", all_documents_similarity_sorted_topXpercent[0][0][0][0])
    
    ## If highest similarity is < say 0.2 then tell the user to add more description for better results? - So as to tackle single word or       line descriptions. To tackle persistant users, add a button if they want analytics with only that much of description?
    if(all_documents_similarity_sorted_topXpercent[0][0][0][0] < 0.35):
        print ("For better analytics, enter more description specific to your app idea")

    ## Print index of the description found to be similar
    #print (all_documents_similarity_sorted_topXpercent[0][1])

    ## Link Datasets and Find Weighted Average of Ratings and other details
    total_weight = 0
    total_weighted_rating = 0
    users_by_rating_dict = { "1.0":0, "1.5":0, "2.0":0, "2.5":0, "3.0":0, "3.5":0, "4.0":0, "4.5":0, "5.0":0 }
    ## For equalized 'number of users' - apple app store lets you rate with integers from 1 - 5 
    users_by_rating_equalized_dict = { "1.0":0, "2.0":0, "3.0":0, "4.0":0, "5.0":0 }
    users_by_ageGroup_dict = { "Children_5-13":0, "Teenager_13-18":0, "Adult_18-50":0, "Elderly_50+":0}
    total_users_that_rated = 0
    ## Arbitrary installs factor value, , through intuition - Apple dataset does not contain information about total installs so have to calculate an average value assuming that every 1 person among 'total_users_that_rated/installs_factor' number of persons rates the app
    installs_factor = 250 ## depends on topXpercent
    
    for i in range(len(all_documents_similarity_sorted_topXpercent)):
        #document_rating = data_full.iloc[(all_documents_similarity_sorted_topXpercent[i][1])]['user_rating']
        document_rating = rating_array[(all_documents_similarity_sorted_topXpercent[i][1])]
        document_rating_count = rating_count_array[(all_documents_similarity_sorted_topXpercent[i][1])]
        if document_rating_count == 0:
            continue
        if document_rating == 0:
            continue
        document_name = track_name_array[(all_documents_similarity_sorted_topXpercent[i][1])]
        document_id = all_documents_similarity_sorted_topXpercent[i][1]
        #if (i<=3):
            #print ("id:",document_id, "name:", document_name, "rating:", document_rating, "rating_count:", document_rating_count, "similarity:", all_documents_similarity_sorted_topXpercent[i][0][0][0])
        
        
        ## Find the final Average rating - Weighted average of ratings of topXpercent similar documents 
        ## Considering Document weight = Similarity Score multiplied by document_rating_count
        document_weight = all_documents_similarity_sorted_topXpercent[i][0][0][0]*document_rating_count
        document_weighted_rating = document_weight*document_rating
        total_weighted_rating = total_weighted_rating+document_weighted_rating
        total_weight = total_weight+document_weight
        
        
        ## For the actual graph of "number of users" by "Rating Given"
        this_rating = str(document_rating)
        users_at_this_rating = users_by_rating_dict[this_rating]
        users_by_rating_dict[this_rating] = int(users_at_this_rating + document_weight)
        ## For the equalized graph of "number of users" by "Rating Given"
        ## Equalizing the number of user per rating:
        ## According to the 5 diff ratings, we make 5 diff dict of percentages of usage for the 5 ratings
        ## Arbitrary average percentage values, through intuition- Apple dataset does not contain information about total users per rating give, for each app. Therefore we use arbitrary average values             
        users_by_rating_equalized_dict = users_by_rating_equalized_dict_modify_by_percentage(this_rating, users_by_rating_equalized_dict, document_weight)
        
        ## Keeping count of total_users_that_rated a particular app
        total_users_that_rated = total_users_that_rated + document_weight
        
        
        ## For the graph of "number of users" by "Age Group" -  
        ## According to the 4 diff content ratings, we make 4 diff dict of percentages of usage for the 4 age groups
        ## Arbitrary average percentage values, through intuition- Apple dataset does not contain information about total user rating per age group or total installs per age group, for each app. Therefore we use arbitrary average values
        age_group = age_group_array[(all_documents_similarity_sorted_topXpercent[i][1])]          
        users_by_ageGroup_dict = users_by_ageGroup_dict_modify_by_percentage(age_group, users_by_ageGroup_dict, document_weight)
        
        #print ("users:", document_weight, "rating:", document_rating, "previous users at this rating:", users_at_this_rating, "new users at this rating", users_by_rating_dict[this_rating])
        #print ("users:", document_weight, "rating:", document_rating, "Content rating:", age_group)
        
        #print ("\nCurrent users_by_ageGroup_dict:", users_by_ageGroup_dict)
     
    
    ## Final Average rating    
    final_rating = total_weighted_rating/total_weight
    
    print ("For WHATSAPP App description:")
    print ("\nAfter the user has entered the description...")
    print ("\n\n------------------------------------------------------------------------------------")
    print ("\n\nWEBPAGE 1: General Overview")
    
    if (final_rating >= 3): 
        print ("\nSelling Ability: Excellent")
    else:
        print ("\nThe app will be a failure :(")
    ## Total predicted installs
    
    ## Print rounded off final rating
    print ("\nPredicted rating: ", round(final_rating, 2)) ## 2 decimal places

    
    
    ## Print genre as the genre of the description with the highest similarity
    prime_genre = genre[all_documents_similarity_sorted_topXpercent[0][1]]
    print("\nDetected Genre: ", prime_genre)
    
    factor = total_users_that_rated/installs_factor
    print("\nTotal installs: ", int(total_users_that_rated*factor), "   <Link to WEBPAGE 3>")
    
    print("\nTotal users that are likely to rate the app on app store: ", int(total_users_that_rated), "   <Link to WEBPAGE 2>")
    
    
    print ("\n\n------------------------------------------------------------------------------------")
    print ("\n\nWEBPAGE 2: Insights on User Ratings")
    
    ## Total users by rating - actual (output not modified): 
    #print("\nUsers by rating - actual:", users_by_rating_dict)
    ## total_users_that_rated as the total number of ratings given
    
    ## Total users by rating - equalized (output modified):
    ## users_by_rating_dict dictionary is not equalized, i.e it could be possible that:
    ## Consider the following final users_by_rating_dict dictionary - Users by rating: {'1.0': 0, '1.5': 0, '2.0': 0, '2.5': 0, '3.0': 503946, '3.5': 0, '4.0': 27265, '4.5': 9559, '5.0': 0}
    ## There are no users at rating 1.0, 2.0, and so on which will not give a distributed graph
    ## Therefore we have to equalize the graph to some extent so that the peaks get distributed and we get a smoother bar graph (This is possibly manipulation of dataset but the kaggle dataset does not have user count for each rating increment for any particular distribution, and that's why we have to normalize the graph)    
    ## NOTE - Could not find a library for this so make a function for equalization?
    print ("\nGraph of No. of Users by Rating:", users_by_rating_equalized_dict)
    
    
    print ("\n\n------------------------------------------------------------------------------------")
    print ("\n\nWEBPAGE 3: Insights on Installs by Age Group")
    ## Total predicted "users that rated" ordered by age
    #print ("\nUsers that rated, ordered by age:", users_by_ageGroup_dict)
    
    ## Total predicted "installs" ordered by age 
    for x in users_by_ageGroup_dict:
        temp4 = users_by_ageGroup_dict[x]
        users_by_ageGroup_dict[x] = int(temp4*factor)
        
    print ("\nGraph of installs by Age Group:", users_by_ageGroup_dict)
    
    
    print ("\n\n------------------------------------------------------------------------------------")
    print ("\n\nIN ALL OF THE 3 WEBPAGES, WE CAN SHOW AT THE BOTTOM: Top 3 similar apps on the app store")
    for i in range(0, 3):
        document_name = track_name_array[(all_documents_similarity_sorted_topXpercent[i][1])]
        document_rating = rating_array[(all_documents_similarity_sorted_topXpercent[i][1])]
        document_rating_count = rating_count_array[(all_documents_similarity_sorted_topXpercent[i][1])]
        this_document_installs = document_rating_count*(document_rating_count/installs_factor)
        #print ("\n", i+1, "-", document_name, "  Rating:", document_rating, "  Installs:", int(this_document_installs) , "  Similarity Score:", round(all_documents_similarity_sorted_topXpercent[i][0][0][0]*100), "%")
        print ("\n", i+1, "-", document_name, "  Rating:", document_rating, "  Similarity Score:", round(all_documents_similarity_sorted_topXpercent[i][0][0][0]*100), "%")
        print ("<1 or 2 lines of the description of this similar app>")
        
    print ("\n\n For further insights and more similar apps - Sign up/Login     <Link to Sign Up/Login Page>" )
        
    return final_rating
   
    
        
#print ("FILES LOADED")


start_time = time.time()

test_description = "WhatsApp Messenger is a FREE messaging app available for Android and other smartphones. WhatsApp uses your phone's Internet connection (4G/3G/2G/EDGE or Wi-Fi, as available) to let you message and call friends and family. Switch from SMS to WhatsApp to send and receive messages, calls, photos, videos, documents, and Voice Messages. WHY USE WHATSAPP: • NO FEES: WhatsApp uses your phone's Internet connection (4G/3G/2G/EDGE or Wi-Fi, as available) to let you message and call friends and family, so you don't have to pay for every message or call.* There are no subscription fees to use WhatsApp. • MULTIMEDIA: Send and receive photos, videos, documents, and Voice Messages. • FREE CALLS: Call your friends and family for free with WhatsApp Calling, even if they're in another country.* WhatsApp calls use your phone's Internet connection rather than your cellular plan's voice minutes. (Note: Data charges may apply. Contact your provider for details. Also, you can't access 911 and other emergency service numbers through WhatsApp). • GROUP CHAT: Enjoy group chats with your contacts so you can easily stay in touch with your friends or family. • WHATSAPP WEB: You can also send and receive WhatsApp messages right from your computer's browser. • NO INTERNATIONAL CHARGES: There's no extra charge to send WhatsApp messages internationally. Chat with your friends around the world and avoid international SMS charges.* • SAY NO TO USERNAMES AND PINS: Why bother having to remember yet another username or PIN? WhatsApp works with your phone number, just like SMS, and integrates seamlessly with your phone's existing address book. • ALWAYS LOGGED IN: With WhatsApp, you're always logged in so you don't miss messages. No more confusion about whether you're logged in or logged out. • QUICKLY CONNECT WITH YOUR CONTACTS: Your address book is used to quickly and easily connect you with your contacts who have WhatsApp so there's no need to add hard-to-remember usernames. • OFFLINE MESSAGES: Even if you miss your notifications or turn off your phone, WhatsApp will save your recent messages until the next time you use the app. • AND MUCH MORE: Share your location, exchange contacts, set custom wallpapers and notification sounds, email chat history, broadcast messages to multiple contacts at once, and more! *Data charges may apply. Contact your provider for details."




if detect(test_description) != 'en':
    print ("Error: Please enter plain english detailed description")
    exit()
else:
    ## POS Tagging and Lemmatization
    lemmatized_Description = lemmatizeDescription(test_description)
    #print ("HI")

test_description_modified = lemmatized_Description
#print (test_description_modified)


## SORT ONLY THOSE MUCH SIMILARITIES THAT ARE NEEDED - all_documents_similarity_sorted - dont sort all 7000 entires using the inbuilt sorted function - Tried this, did not significantly change the runtime
## LSH HASHING - BUCKETS 8-10 
## Sklearn KNN
## WHAT IF WE REMOVE POS and LEMMATIZATION PART AND DIRECTLY PASS test_description in find_similarity()
## WHAT IF WE USE inbuilt KNN function instead of COSINE SIMILARITY
## WHAT IF WE REDUCE VOCAB SIZE TO 7000 or something like that - Tried this, it largely reduced the runtime with negligible change in accuracy. Also, try Text summarization? - Might decrease accuracy but also decrease running time

final_rating = find_similarity(test_description_modified)


#print("\n--- %s seconds ---" % (time.time() - start_time))

For WHATSAPP App description:

After the user has entered the description...


------------------------------------------------------------------------------------


WEBPAGE 1: General Overview

Selling Ability: Excellent

Predicted rating:  4.11

Detected Genre:  Social Networking

Total installs:  844283883    <Link to WEBPAGE 3>

Total users that are likely to rate the app on app store:  459424    <Link to WEBPAGE 2>


------------------------------------------------------------------------------------


WEBPAGE 2: Insights on User Ratings

Graph of No. of Users by Rating: {'1.0': 5538, '2.0': 17329, '3.0': 61256, '4.0': 190216, '5.0': 173988}


------------------------------------------------------------------------------------


WEBPAGE 3: Insights on Installs by Age Group

Graph of installs by Age Group: {'Children_5-13': 41884822, 'Teenager_13-18': 211063340, 'Adult_18-50': 464671237, 'Elderly_50+': 126632123}


-------------------------------------------------------------------